Use PySCF directly to manually generate the second quantized form of the H2 molecule. Then use OpenFermion package to do the same. The purpose is to understand how the calculation is performed in the OpenFermion Package.

In [2]:
import numpy as np
from scipy import linalg, sparse
from itertools import product
from pyscf import gto, scf, ao2mo
import openfermionpyscf as ofpyscf
from openfermion.ops import FermionOperator
from openfermion import utils, ops
import openfermion as opf

import tqdm
from quantum.utils import plotting as PLT
from quantum import constants

Build H2 molecule using the `gto` module. Solve it using the restricted Hartree Fock method.

In [3]:
bond_length = 0.7 # Angstrom
mol = gto.Mole()
mol.build(atom=f'H 0 0 {-bond_length/2};H 0 0 {bond_length/2}',basis='sto-3g',unit='A')
sol = scf.RHF(mol).run()
print('Coeffs:\n',sol.mo_coeff)
Hcore = sol.get_hcore()
J,K   = sol.get_jk()
F     = sol.get_fock()
S     = sol.get_ovlp()

converged SCF energy = -1.11734903499028
Coeffs:
 [[ 0.54455869  1.26206594]
 [ 0.54455869 -1.26206594]]


In [4]:
eri=ao2mo.get_ao_eri(mol)
print('AO repulsion integrals:\n',eri)

ori=ao2mo.get_mo_eri(eri,sol.mo_coeff)
print('\nMO repulsion Integrals:\n',ori)

eri_full = ao2mo.full(mol, np.eye(2))
print('Full:\n',eri_full)

ori_full=ao2mo.full(mol, sol.mo_coeff)
print('\nFull MO:\n',ori_full)

Vtensor = ao2mo.restore(1, ao2mo.kernel(mol, sol.mo_coeff), sol.mo_coeff.shape[1])
# <pq | V | rs > = (ps|qr) = pyscfERI[p,s,q,r]
Vtensor = Vtensor.transpose((0,2,3,1))
print('\nTensor Form:\n',Vtensor)

AO repulsion integrals:
 [[0.77460594 0.46762065 0.58512264]
 [0.46762065 0.324858   0.46762065]
 [0.58512264 0.46762065 0.77460594]]

MO repulsion Integrals:
 [[6.82389533e-01 6.90428546e-17 6.70732778e-01]
 [1.50017487e-16 1.79000576e-01 3.45644368e-16]
 [6.70732778e-01 8.07231136e-17 7.05105632e-01]]
Full:
 [[0.77460594 0.46762065 0.58512264]
 [0.46762065 0.324858   0.46762065]
 [0.58512264 0.46762065 0.77460594]]

Full MO:
 [[6.82389533e-01 6.90428546e-17 6.70732778e-01]
 [1.50017487e-16 1.79000576e-01 3.45644368e-16]
 [6.70732778e-01 8.07231136e-17 7.05105632e-01]]

Tensor Form:
 [[[[6.82389533e-01 1.50017487e-16]
   [6.90428546e-17 1.79000576e-01]]

  [[6.90428546e-17 1.79000576e-01]
   [6.70732778e-01 3.45644368e-16]]]


 [[[1.50017487e-16 6.70732778e-01]
   [1.79000576e-01 8.07231136e-17]]

  [[1.79000576e-01 8.07231136e-17]
   [3.45644368e-16 7.05105632e-01]]]]


In [5]:
X = sol.mo_coeff 
Hcore = X.T @ sol.get_hcore() @  X
Vtensor = ao2mo.restore(1, ao2mo.kernel(mol, X), X.shape[1]).transpose((0,2,3,1))

spatial_index = np.array([0,0,1,1])
H0 = sol.energy_nuc()
for i,j in product([0,1],[0,1]):
   H0 += Hcore[i,j]*FermionOperator( [ (2*i,1), (2*j,0) ] )
   H0 += Hcore[i,j]*FermionOperator( [ (2*i+1,1), (2*j+1,0) ] )
#
Vc = 0
for p,q,r,s in product([0,1],[0,1],[0,1],[0,1]):
    if np.abs(Vtensor[p,q,r,s])>1e-8:
        Vc += (
            Vtensor[p,q,r,s]/2*FermionOperator( ( (2*p, 1), (2*q, 1), (2*r, 0), (2*s, 0) ) ) + 
            Vtensor[p,q,r,s]/2*FermionOperator( ( (2*p, 1), (2*q+1, 1), (2*r+1, 0), (2*s, 0) ) ) + 
            Vtensor[p,q,r,s]/2*FermionOperator( ( (2*p+1, 1), (2*q, 1), (2*r, 0), (2*s+1, 0) ) ) + 
            Vtensor[p,q,r,s]/2*FermionOperator( ( (2*p+1, 1), (2*q+1, 1), (2*r+1, 0), (2*s+1, 0) ) ) 
        )

HJW = opf.jordan_wigner(H0 + Vc)
H = opf.linalg.get_sparse_operator(HJW)
E, _ = sparse.linalg.eigsh(H, k=1, which='SA')
print('Ground State Energy: ',E)


Ground State Energy:  [-1.13618945]


In [6]:
HJW = opf.jordan_wigner(H0 + Vc)
H = opf.linalg.get_sparse_operator(HJW)
E, _ = sparse.linalg.eigsh(H, k=1, which='SA')
print('Ground State Energy: ',E)
#print('JW form\n',HJW)

Ground State Energy:  [-1.13618945]


In [7]:
HBK = opf.bravyi_kitaev(H0 + Vc)
H = opf.linalg.get_sparse_operator(HBK)
E,_ = sparse.linalg.eigsh(H, k=1, which='SA')
print('Ground State Energy: ',E)
print('JBK form\n',HBK)

Ground State Energy:  [-1.13618945]
JBK form
 (-0.042078976477822064+0j) [] +
(0.04475014401535165+0j) [X0 Z1 X2] +
(0.04475014401535165+0j) [X0 Z1 X2 Z3] +
(0.04475014401535165+0j) [Y0 Z1 Y2] +
(0.04475014401535165+0j) [Y0 Z1 Y2 Z3] +
(0.17771287465139937+0j) [Z0] +
(0.17771287465139937+0j) [Z0 Z1] +
(0.16768319457718966+0j) [Z0 Z1 Z2] +
(0.16768319457718966+0j) [Z0 Z1 Z2 Z3] +
(0.12293305056183804+0j) [Z0 Z2] +
(0.12293305056183804+0j) [Z0 Z2 Z3] +
(0.17059738328801052+0j) [Z1] +
(-0.242742805131405+0j) [Z1 Z2 Z3] +
(0.17627640804319608+0j) [Z1 Z3] +
(-0.242742805131405+0j) [Z2]


In [8]:
geometry = [("H", (0.0, 0.0, -bond_length/2)), ("H", (0.0, 0.0, bond_length/2))]
molecule = opf.MolecularData(geometry,'6-31g',multiplicity=1,charge=0)
molecule = ofpyscf.run_pyscf(molecule, run_scf=False, run_cisd=False,run_ccsd=False, run_fci=False, verbose=True)
#generate_molecular_hamiltonian(geometry, 'sto-3g', multiplicity=1, charge=0)


Hartree-Fock energy for H2_6-31g_singlet (2 electrons) is -1.1261231613839136.


In [9]:
molecule.two_body_integrals

array([[[[ 6.65568000e-01,  9.74508280e-19,  1.72481025e-01,
          -1.20943990e-16],
         [-4.30934749e-17,  7.56290624e-02,  4.08414278e-17,
           7.83652787e-02],
         [ 1.72481025e-01,  5.43855950e-17,  1.11349802e-01,
           8.85255004e-18],
         [-1.07530531e-16,  7.83652787e-02,  2.58942317e-17,
           1.41915120e-01]],

        [[-4.30934749e-17,  7.56290624e-02,  4.08414278e-17,
           7.83652787e-02],
         [ 4.32259761e-01, -1.31221886e-16,  4.70975257e-02,
           6.51471068e-17],
         [-1.60777378e-17, -2.26170547e-02,  1.70503680e-17,
           1.72288023e-02],
         [ 1.43379632e-01,  1.03723184e-16,  7.12140873e-02,
          -1.94414433e-16]],

        [[ 1.72481025e-01,  5.43855950e-17,  1.11349802e-01,
           8.85255004e-18],
         [-1.60777378e-17, -2.26170547e-02,  1.70503680e-17,
           1.72288023e-02],
         [ 5.36196381e-01,  9.91638586e-17,  1.19500364e-01,
           8.31909890e-17],
         [-6.5560

## Directly using OpenFermion

In [10]:
# Set molecule parameters
geometry = [("H", (0.0, 0.0, -bond_length/2)), ("H", (0.0, 0.0, bond_length/2))]
hamiltonian = ofpyscf.generate_molecular_hamiltonian(geometry, 'sto-3g', multiplicity=1, charge=0)
hamiltonian_ferm_op = opf.get_fermion_operator(hamiltonian)

In [11]:
hamiltonian_jw = opf.jordan_wigner(hamiltonian_ferm_op)
hamiltonian_jw_sparse = opf.get_sparse_operator(hamiltonian_jw)
eigs, _ = sparse.linalg.eigsh(hamiltonian_jw_sparse, k=1, which="SA")
ground_energy = eigs[0]
print("Ground_energy: {}".format(ground_energy))
print("JWT transformed Hamiltonian:")
print(hamiltonian_jw)

Ground_energy: -1.13618945406592
JWT transformed Hamiltonian:
(-0.042078976477822036+0j) [] +
(-0.04475014401535165+0j) [X0 X1 Y2 Y3] +
(0.04475014401535165+0j) [X0 Y1 Y2 X3] +
(0.04475014401535165+0j) [Y0 X1 X2 Y3] +
(-0.04475014401535165+0j) [Y0 Y1 X2 X3] +
(0.17771287465139934+0j) [Z0] +
(0.17059738328801052+0j) [Z0 Z1] +
(0.12293305056183804+0j) [Z0 Z2] +
(0.16768319457718966+0j) [Z0 Z3] +
(0.17771287465139937+0j) [Z1] +
(0.16768319457718966+0j) [Z1 Z2] +
(0.12293305056183804+0j) [Z1 Z3] +
(-0.242742805131405+0j) [Z2] +
(0.17627640804319608+0j) [Z2 Z3] +
(-0.242742805131405+0j) [Z3]


In [12]:
hamiltonian_bk = opf.bravyi_kitaev(hamiltonian_ferm_op)
hamiltonian_bk_sparse = opf.get_sparse_operator(hamiltonian_bk)
eigs, _ = sparse.linalg.eigsh(hamiltonian_bk_sparse, k=1, which="SA")
ground_energy = eigs[0]
print("Ground_energy: {}".format(ground_energy))
print("JWT transformed Hamiltonian:")
print(hamiltonian_bk)

Ground_energy: -1.1361894540659245
JWT transformed Hamiltonian:
(-0.042078976477822036+0j) [] +
(0.04475014401535165+0j) [X0 Z1 X2] +
(0.04475014401535165+0j) [X0 Z1 X2 Z3] +
(0.04475014401535165+0j) [Y0 Z1 Y2] +
(0.04475014401535165+0j) [Y0 Z1 Y2 Z3] +
(0.17771287465139934+0j) [Z0] +
(0.17771287465139937+0j) [Z0 Z1] +
(0.16768319457718966+0j) [Z0 Z1 Z2] +
(0.16768319457718966+0j) [Z0 Z1 Z2 Z3] +
(0.12293305056183804+0j) [Z0 Z2] +
(0.12293305056183804+0j) [Z0 Z2 Z3] +
(0.17059738328801052+0j) [Z1] +
(-0.242742805131405+0j) [Z1 Z2 Z3] +
(0.17627640804319608+0j) [Z1 Z3] +
(-0.242742805131405+0j) [Z2]
